## Imports

In [1]:
import os
import re
import glob
import json
import string
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
%matplotlib inline

## Read & Combine Data

### LinkedIn Job Postings

In [2]:
listings_li = pd.read_csv("./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/MLE Jobs LinkedIn.csv")
listings_li.head(3)

,Unnamed: 0,Job Title,Company Name,Location,Work Mode,Employment Type,Company Headcount,Industry,JD,JobID
0,0,SENIOR COMPUTER VISION DATA SCIENTIST,80 Acres Farms,"Hamilton, OH",On-site,Full-time · Mid-Senior level,201-500 employees,Farming,About The Company\r\n\r\nThe Future of Fresh i...,3331975665
1,1,Senior Software Engineer,Terran Robotics,"Bloomington, IN",Hybrid,Full-time,1-10 employees,None,Terran Robotics is building construction robot...,2841811846
2,2,Systems Engineer,ZF Group,Greater Lansing,Hybrid,Full-time · Mid-Senior level,"10,001+ employees",Motor Vehicle Manufacturing,Job description:\r\n\r\nJob Description:\r\n\r...,3369017400


### Glassdoor Job Postings

In [3]:
path_folder = "./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings"
paths_csv = glob.glob(os.path.join(path_folder, "*.csv"))
paths_csv

['./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/MLE Jobs LinkedIn.csv',
 './Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/DataScientist.csv',
 './Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/DataAnalyst.csv',
 './Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/DataEngineer.csv',
 './Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/BusinessAnalyst.csv']

In [4]:
cols = ['Job Title',
         'Salary Estimate',
         'Job Description',
         'Rating',
         'Company Name',
         'Location',
         'Headquarters',
         'Size',
         'Founded',
         'Type of ownership',
         'Industry',
         'Sector',
         'Revenue',
         'Competitors',
         'Easy Apply']

In [5]:
listings_gd = pd.DataFrame()

for file in paths_csv:
    if "MLE Jobs LinkedIn" in file:
        continue
        
    jobfamily = file.split("/")[-1].split(".csv")[0] # Extract job family from file path
    print(jobfamily)
    
    df = pd.read_csv(file)
    df = df[cols] # Reorder & subset cols for data integrity
    listings_gd = pd.concat([listings_gd, df], axis=0, ignore_index=True)

del df
listings_gd.head(3)

DataScientist
DataAnalyst
DataEngineer
BusinessAnalyst


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True


## Merge Datasets

In [6]:
cols = ["Company Name", "Job Title", "Location", "Industry", "Job Description"]

listings_li = listings_li.rename(columns={"JD": "Job Description"})
listings = pd.concat([listings_li[cols], listings_gd[cols]], axis=0, ignore_index=True)

In [7]:
listings.shape

(16717, 5)

## Data Cleaning

### Companies

In [8]:
listings['Company Name'] = listings['Company Name'].apply(lambda x: str(x).split("\n")[0])

### Job Titles

In [9]:
def match_job_title(job_title, keywords):
    # Check if job title is present in a predefined list of titles
    if job_title in keywords:
        return 1
    return 0

In [10]:
with open("./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/job_mapping.json", 'r') as f:
    job_mapping = json.load(f)

for jobtitle, keywords in job_mapping.items():
    listings[jobtitle] = listings["Job Title"].apply(lambda title: match_job_title(title, keywords))

In [11]:
listings = listings.dropna(subset=['Job Title', 'Job Description'])

In [12]:
listings.head(3)

,Company Name,Job Title,Location,Industry,Job Description,Business Analyst,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer,Research
0,80 Acres Farms,SENIOR COMPUTER VISION DATA SCIENTIST,"Hamilton, OH",Farming,About The Company\r\n\r\nThe Future of Fresh i...,0,0,0,1,1,0
1,Terran Robotics,Senior Software Engineer,"Bloomington, IN",None,Terran Robotics is building construction robot...,0,0,0,0,0,0
2,ZF Group,Systems Engineer,Greater Lansing,Motor Vehicle Manufacturing,Job description:\r\n\r\nJob Description:\r\n\r...,0,0,0,0,0,0


## Feature Extraction from JDs

### Entities of interest: 

#### Skills
1. Programming Languages (exp- Python, SQL, R)
2. ML Frameworks (exp- Sklearn, Tensorflow, Pytorch)
3. ML Algorithms (exp- Regression, DTs, GBMs, GNNs)
4. Visualization Libraries (exp- Matplotlib, Seaborn)
5. Cloud Platforms (exp- AWS, Azure, GCP)
6. Databases (exp- MySQL, MongoDB)
7. BI Tools (exp- Tableau, PowerBI)

#### Job Requirements
1. Years of experience
2. Degree (Bachelor's, Master's, PhD)

### Dictionary based mapping of skills

In [13]:
with open("./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/ds_survey/survey_cols.json", 'r') as f:
    survey_cols = json.load(f)

[col for col in survey_cols if 'MLFramework_' in col]

['MLFramework_Scikit-learn',
 'MLFramework_TensorFlow',
 'MLFramework_Keras',
 'MLFramework_PyTorch',
 'MLFramework_Fast.ai',
 'MLFramework_Xgboost',
 'MLFramework_LightGBM',
 'MLFramework_CatBoost',
 'MLFramework_Caret',
 'MLFramework_Tidymodels',
 'MLFramework_JAX',
 'MLFramework_PyTorch Lightning',
 'MLFramework_Huggingface',
 'MLFramework_None',
 'MLFramework_Other',
 'MLFramework_MXNet',
 'MLFramework_Prophet',
 'MLFramework_H2O 3']

In [14]:
def get_kw_dict(kw, survey_cols):
    """
    Get dict of col names which match the given keyword.
    Exp: kw='MLFramework' returns
        {'MLFramework_Xgboost': ['Xgboost'], 'MLFramework_TensorFlow': ['Tensorflow']} ...
    """
    return {col: [" ".join(col.split('_')[1:]).strip()] for col in survey_cols if kw in col}

In [15]:
prog_languages = get_kw_dict("ProgLanguage_", survey_cols)
ml_frameworks = get_kw_dict("MLFramework_", survey_cols)
ml_algos = get_kw_dict("MLAlgo_", survey_cols)
viz_libs = get_kw_dict("VizLib_", survey_cols)
cloud_providers = get_kw_dict("CloudProviderUsed_", survey_cols)
dbs = get_kw_dict("DBUsed_", survey_cols)
bi_tools = get_kw_dict("BIToolUsed_", survey_cols)

In [16]:
# Manual validation of keywords

prog_languages['ProgLanguage_R'] = [' R ']
prog_languages['ProgLanguage_C'] = [' C ']
prog_languages['ProgLanguage_Go'] = [' Go ']

ml_frameworks['MLFramework_Scikit-learn'].extend(['sklearn'])

ml_algos['MLAlgo_Linear or Logistic Regression'].extend(['regression'])
ml_algos['MLAlgo_Decision Trees or Random Forests'].extend(['decision tree', 'random forest'])
ml_algos['MLAlgo_Gradient Boosting Machines (xgboost, lightgbm, etc)'].extend(['xgboost', 'lightgbm', 'gbm', 'gradient boost'])
ml_algos['MLAlgo_Bayesian Approaches'].extend(['bayes'])
ml_algos['MLAlgo_Evolutionary Approaches'].extend(['genetic algo'])
ml_algos['MLAlgo_Dense Neural Networks (MLPs, etc)'].extend(['dnn', 'deep neural net', 'perceptron'])
ml_algos['MLAlgo_Convolutional Neural Networks'].extend(['cnn', 'yolo', 'resnet', 'vgg', 'inception'])
ml_algos['MLAlgo_Generative Adversarial Networks'].extend([' gan '])
ml_algos['MLAlgo_Recurrent Neural Networks'].extend(['rnn', 'lstm', 'gru'])
ml_algos['MLAlgo_Transformer Networks (BERT, gpt-3, etc)'].extend(['bert', 'gpt', 'transformer', 'universal sentence encoder'])
ml_algos['MLAlgo_Autoencoder Networks (DAE, VAE, etc)'].extend(['dae', 'vae', 'autoencoder'])
ml_algos['MLAlgo_Graph Neural Networks'].extend(['gnn', 'graph neural net'])

viz_libs['VizLib_Plotly / Plotly Express'].extend(['plotly'])
viz_libs['VizLib_Ggplot / ggplot2'].extend(['ggplot'])
viz_libs['VizLib_D3 js'].extend(['d3'])

cloud_providers['CloudProviderUsed_Amazon Web Services (AWS)'].extend(['AWS'])
cloud_providers['CloudProviderUsed_Microsoft Azure'].extend(['Azure'])
cloud_providers['CloudProviderUsed_Google Cloud Platform (GCP)'].extend(['google cloud', 'gcp'])
cloud_providers['CloudProviderUsed_IBM Cloud / Red Hat'].extend(['red hat', 'ibm cloud'])

dbs['DBUsed_Microsoft SQL Server'].extend(['Mircosoft SQL'])
dbs['DBUsed_Amazon Redshift'].extend(['Redshift'])
dbs['DBUsed_Amazon RDS'].extend([' RDS '])
dbs['DBUsed_Amazon DynamoDB'].extend(['DynamoDB'])
dbs['DBUsed_Google Cloud BigQuery'].extend(['BigQuery'])

bi_tools['BIToolUsed_Amazon QuickSight'].extend(['QuickSight'])
bi_tools['BIToolUsed_Microsoft Power BI'].extend(['Power BI'])

In [17]:
all_features = {**prog_languages, **ml_frameworks, **ml_algos, **viz_libs, **cloud_providers, **dbs, **bi_tools}
all_features = {k:v for k,v in all_features.items() if not ('other' in k.lower() or 'none' in k.lower())}

In [18]:
regex_punct = re.compile('[%s]' % re.escape(string.punctuation))

def remove_punct(text):
    cleaned = regex_punct.sub(' ', text) # Replace punctuations with spaces. Exp: "AI/ML" -> "AI ML"
    return re.sub(' +', ' ', cleaned) # Replace multiple spaces with single space

In [19]:
def match_kw(jd, kws):
    "Returns 1 if the JD contains a keyword associated to a skill"
    for kw in kws:
        if kw.lower() in jd.lower():
            return 1
    return 0

In [20]:
listings['Job Description'] = listings['Job Description'].apply(remove_punct)

In [ ]:
%%time
# Add features to listings
for feature, kws in all_features.items():
    listings[feature] = listings['Job Description'].apply(lambda jd: match_kw(jd, kws))

listings.head(3)

In [ ]:
listings['ProgLanguage_Java'].sum()

In [ ]:
listings.shape

# Our Tips to get hired!

### Top Skills (overall) for Data Science jobs

In [ ]:
cnts = listings.iloc[:,11:].sum()
cnts = cnts/len(listings)

# Overall Ranking
cnts.sort_values(ascending=False)[:5]

### Top Programming Languages

In [ ]:
cnts[cnts.index.str.contains("ProgLanguage")].sort_values(ascending=False)[:5]

### Top ML algorithms to add to your resume

In [ ]:
cnts[cnts.index.str.contains("MLAlgo")].sort_values(ascending=False)[:10]

### Top Cloud tools used in the industry

In [ ]:
cnts[cnts.index.str.contains("CloudProviderUsed")].sort_values(ascending=False)[:5]

### Top Databases you should skill yourself in

In [ ]:
cnts[cnts.index.str.contains("DBUsed")].sort_values(ascending=False)[:5]

**Pro tip**: Add Regression & CNNs to your resumes if you haven't already!

### Top Viz tools

In [ ]:
cnts[cnts.index.str.contains("VizLib")].sort_values(ascending=False)[:5]

### Rarely present tools

In [ ]:
cnts[cnts<=0.001].sort_values()[:20]

In [ ]:
# Export data
listings.to_csv("Joblistings.csv", index=False)

#### To Do: NER model for skills extraction (so that we are not limited by our list of skills)
https://arxiv.org/pdf/2101.11431.pdf

### Experience (Yrs)

In [ ]:
# To Do (If formulating hypothesis)

### Degree

In [ ]:
# To Do (If formulating hypothesis)

## Dataset Characteristics

### Companies in our Dataset

In [ ]:
fig = plt.figure(figsize=(12,8))

gs = GridSpec(2, 2, figure=fig, wspace=0.75, hspace=0.4, width_ratios=[1,1])

ax1 = fig.add_subplot(gs[:, 1])
ax2 = fig.add_subplot(gs[0, 0])
ax3 = fig.add_subplot(gs[1, 0])

companies = listings['Company Name'].value_counts()[:20] # Top 20 companies
sns.barplot(companies.values, companies.index, palette='mako', ax=ax1)
ax1.set_title('Top 20 Companies by Job postings', size=16, pad=7)
ax1.set_xlabel('No. of Job postings', fontsize=12, labelpad=7)

industries = listings['Industry'].value_counts()[:10] # Top 10 Industries
industries = industries[industries.index!="-1"]
sns.barplot(industries.values, industries.index, palette='mako', ax=ax2)
ax2.set_title('Top 10 Industries by Job postings', size=16, pad=10)
ax2.set_xlabel('No. of Job postings', fontsize=12, labelpad=7)

locations = listings['Location'].value_counts()[:10] # Top 10 Industries
sns.barplot(locations.values, locations.index, palette='mako', ax=ax3)
ax3.set_title('Top 10 Locations by Job postings', size=16, pad=7)
ax3.set_xlabel('No. of Job postings', fontsize=12, labelpad=7)

plt.show()